In [82]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import ascii



In [83]:
# 1 - load data

data = ascii.read("tableb2.pos")  
radec=data.as_array()

# 2 - create coord list

ra=data['col1']
dec=data['col2']
start=int(20000)
stop=int(start+50)
coord_list = SkyCoord(ra=ra[start:stop], dec=dec[start:stop], unit=u.deg)


In [84]:
# 3 - Query Vizier, method 1

result = Vizier.query_region(coord_list, radius=0.2 * u.arcsec, catalog='II/246')

# Im interested only in the IDs, print them:

#result[0]['_2MASS']

# Not bad at all but this will time out for large lists such as the >70000 records we have here
# => can be done in chunks of ~1000 for instance
# One drawback of this method is all the default columns of the catalog are retrieved
# whereas we are only interested in "_2MASS', so we're waiting bandwidth
# => Can we improve things (the size of the chunk manageable for isntance)
# By requesting only the "_2MASS" column is sent back?


In [81]:
# 4 Query Vizier, method 2

# The columns to be reteieved are specified in the VizieR instanciation

v=Vizier(columns=["2MASS"], catalog="II/246")
result = v.query_region(coord_list, radius=0.2 * u.arcsec)

# But sadly this bugs


ReadTimeout: HTTPConnectionPool(host='vizier.u-strasbg.fr', port=80): Read timed out. (read timeout=60)

In [89]:
# The original example from Katharina
ra = [120.689, 150.3684, 4.9516]
dec = [-23.4578, -65.794236, 35.5631]
coord_list = SkyCoord(ra=ra, dec=dec, unit=u.deg)
catalog_list = Vizier.find_catalogs('2MASS PSC')
print({k:v.description for k,v in catalog_list.items()})
result = Vizier.query_region(coord_list, radius=0.2 * u.deg, catalog='II/246')
print(result[0])


{'J/ApJS/184/138': 'XID II: RASS/BSC-2MASS/PSC cross-association (Haakonsen+, 2009)', 'II/281': '2MASS 6X Point Source Working Database / Catalog (Cutri+ 2006)', 'II/246': '2MASS All-Sky Catalog of Point Sources (Cutri+ 2003)'}
 _q  RAJ2000    DEJ2000        _2MASS       Jmag  ... Rflg Bflg Cflg Xflg Aflg
       deg        deg                       mag   ...                         
--- ---------- ---------- ---------------- ------ ... ---- ---- ---- ---- ----
  1 120.605432 -23.634083 08022530-2338026 14.938 ...  222  111  000    0    0
  1 120.604096 -23.638178 08022498-2338174 13.962 ...  222  111  000    0    0
  1 120.609570 -23.634823 08022629-2338053 14.571 ...  222  111  000    0    0
  1 120.610175 -23.639097 08022644-2338207 16.338 ...  222  111  000    0    0
  1 120.621772 -23.643244 08022922-2338356 15.983 ...  220  110  000    0    0
  1 120.618161 -23.634325 08022835-2338035 16.537 ...  202  101  000    0    0
  1 120.619127 -23.646334 08022859-2338468 16.007 ...  222  1